The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### The keyboard command to write in bash format is: Alt Gr + ù

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

```bash
mkdir students
wget https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&e=1
```

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

```bash
grep "PoD" students.csv >> PoD_sudents.csv
grep "Physics" students.csv >> Physics_sudents.csv
```

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

```bash

for letter in {A..Z}; do n=$(grep -c "^$letter" students.csv); echo "There are $n $letter"; done 

```

1\.d Find out which is the letter with most counts.

```bash
#!/bin/bash

mostcount=0
reclet=""

for letter in {A..Z}; do 
	n=$(grep -c "^$letter" students.csv) 
	if [ $mostcount -lt $n ]; then 
	mostcount=$n 
	reclet=$letter
	fi 
done

echo $reclet
```

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
#!/bin/bash

i=1  # group number
n=0  # member number
first_line=true

while read -r line; do

 if $first_line; then
  first_line=false
  continue  # skip the first line
 fi

 if (( n >= 18 )); then 
  ((i++))  # next group
  n=0      # reset member count
 fi
    
 echo "$line" >> ./sgroups/group_$i.csv
 ((n++))  # increment member count

done < students.csv
```

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

```bash
sed -e "s/, / /g" data.csv | grep -v "^#" >> data.txt
```

2\.b How many even numbers are there?

```bash
#!/bin/bash

a=$(grep "" data.txt)
npari=0

for num in $a; do
	if [ $(($num % 2)) -eq 0 ]; then 
		((npari++)) 
	fi
done

echo $npari	
```

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

```bash
#!/bin/bash

threshold=$(echo "scale=6; 100 * sqrt(3) / 2" | bc -l)
gr=0
sm=0


while read -r line; do
 set -- $line
 x=$1
 y=$2
 z=$3
 
 sumq=$(echo $x*$x + $y*$y + $z*$z | bc -l)
 root=$(echo "scale=6; sqrt($sumq)" | bc -l)
 
 is_less=$(echo "$root < $threshold" | bc -l)
 
 if [ $is_less -eq 1 ]; then
  ((gr++))
 
 else
  ((sm++))
 fi

done < data.txt

echo "Greater: $gr"
echo "Smaller: $sm"

```

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
#!/bin/bash

for (( i=1 ; i < ($1+1) ; i++ )); do
 filename="./copies/data_$i.txt"
 > "$filename"   # Create or empty the file
 
 while read -r line; do
  # Select all values in the line
  for value in $line; do
   result=$(echo "scale=2; $value/$i" | bc -l)
   echo -n "$result " >> "$filename" 
  done
  echo "" >> "$filename"   # New line

 done < data.txt
done
```